In [1]:
import sys
import os

cd = os.getcwd()
wd = os.path.dirname(cd)
sys.path.append(os.path.join(wd,'bin'))

import libsbml
import importlib
import amici
import numpy as np
import re
import pandas as pd
from antimony import *
from modules.copyDir import copyDirectory

# Optional packages to import
import amici.plotting
import matplotlib.pyplot as plt
from time import time
import matplotlib as mpl
from modules.RunSPARCED2 import RunSPARCED

mpl.rcParams['figure.dpi'] = 120



In [2]:
sbml_file = "SPARCED.xml"
model_name= sbml_file[0:-4]
model_output_dir = model_name
sys.path.insert(0, os.path.join(cd,model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()

species_all = list(model.getStateIds())


Vn = 1.75E-12
Vc = 5.25E-12

solver = model.getSolver()          # Create solver instance
solver.setMaxSteps = 1e10

ts = 30

model.setTimepoints(np.linspace(0,ts))

In [3]:
STIMligs = [1.0,0.0,0.005,0.0,0.0,0.0,0.0] # EGF, Her, HGF, PDGF, FGF, IGF, INS

STIMligs_id = ['E', 'H', 'HGF', 'P', 'F', 'I', 'INS']



spIn2 = np.array(model.getInitialStates())

for s,sp in enumerate(STIMligs_id):
    spIn2[species_all.index(sp)] = STIMligs[s]


model.setInitialStates(spIn2)

flagD = 1

th = 72

xoutS_all, xoutG_all, tout_all = RunSPARCED(flagD,th,spIn2,[],sbml_file,model)

In [ ]:
#benchmark - one4all
fnameInput2 = 'SPARCEDo4a_v1' # model file name = USER input
sbml_file2 = fnameInput2+'.xml'
model_name2 = sbml_file2[0:-4] 
model_output_dir2 = model_name2 
sys.path.insert(0, os.path.join(cd,model_output_dir2))
model_module2 = importlib.import_module(model_name2)
model2 = model_module2.getModel()

solver2 = model2.getSolver()
solver2.setMaxSteps = 1e10

th2 = 72*3600
model2.setTimepoints(np.linspace(0,th2,1000))

species_all2 = list(model2.getStateIds())


spIn_o4a = np.array(model_module2.getModel().getInitialStates())

for s,sp in enumerate(STIMligs_id):
    spIn_o4a[species_all2.index(sp)] = STIMligs[s]


model2.setInitialStates(spIn_o4a)

sbml_reader = libsbml.SBMLReader()
sbml_doc2 = sbml_reader.readSBML(sbml_file2)
sbml_model2 = sbml_doc2.getModel()


In [ ]:
rdata_o4a = amici.runAmiciSimulation(model2,solver2)
    


xoutS_all5 = rdata_o4a['x']
tout_all5 = rdata_o4a['t']

In [ ]:


def timecourse_compare2(xs1,xsl1,xs2,t2,xsl2,sp,ts=30,xlim=None,y0=None,ylim=None,me=10):
    t1 = np.array(range(0,len(xs1)))*30
    xt1 = xs1[:,species_all.index(sp)]
    xt2 = xs2[:,species_all2.index(sp)]
    # tout = np.array(range(len(xt1)))*ts/3600
    
    plt.plot(t1/3600,xt1,label=xsl1)
    plt.plot(t2/3600,xt2,label=xsl2,ls='None',marker="x",markevery=int(me),color='r')
    plt.ylim(0,max(max(xt1),max(xt2))*1.25)
    if type(ylim)==int or type(ylim)==float:
        plt.ylim(0,ylim)
        if type(y0)==int or type(y0)==float:
            plt.ylim(y0,ylim)
    if type(xlim)==int or type(xlim)==float:
        plt.xlim(0,xlim)
    plt.xlabel('Time (hours)')
    plt.ylabel(sp+' (nM)')
    plt.title('species timecourse')
    plt.legend()
    plt.show()


In [ ]:
#mpl.rcParams['figure.dpi'] = 120


timecourse_compare2(xoutS_all,'Hybrid',xoutS_all5,tout_all5,'Integrated-SBML','Md',me=20)

In [ ]:
#%% Species AUC check

auc_sparced = np.zeros(len(species_all))
auc_one4all = np.zeros(len(species_all))

for i,sp in enumerate(species_all):
    
    auc_sparced[i] = np.trapz(xoutS_all[:,species_all.index(sp)],tout_all)
    auc_one4all[i] = np.trapz(xoutS_all5[:,species_all2.index(sp)],tout_all5)
    

plt.scatter(auc_sparced,auc_one4all,marker=".")
plt.xscale('log')
plt.yscale('log')
plt.xlim(1e-8,1e8)
plt.ylim(1e-8,1e8)
xpoints = ypoints = plt.xlim()
plt.plot(xpoints, ypoints,linestyle='--', color='k', lw=0.75, scalex=False, scaley=False)
plt.show()

In [ ]:
auc_sparced

In [ ]:
np.isnan(xoutS_all5[:,species_all2.index('ppERK')])

In [ ]:
def timecourse(xoutS,sp,species_all):
    x_t = xoutS[:,species_all.index(sp)]
    tout = np.array(range(0,len(x_t)))*30/3600
    
    plt.plot(tout,x_t)
    plt.ylim(0,max(x_t)*1.25)
    plt.xlabel('Time (hours)')
    plt.ylabel(str(sp))
    plt.show()




def timecourse_x(rdata,sp,species_all):
    x_t = rdata['x'][:,species_all.index(sp)]
    tout = rdata['t']/3600
    
    plt.plot(tout,x_t)
    plt.ylim(0,max(x_t)*1.25)
    plt.xlabel('Time (hours)')
    plt.ylabel(str(sp))
    plt.show()

In [ ]:
timecourse(xoutS_all,'cPARP',species_all)

In [ ]:
timecourse_x(rdata_o4a,'cPARP',species_all2)

In [ ]:
model2.getFixedParameterById('k2730_5')

In [ ]:
model_module.getModel().getInitialStates()[species_all.index('pRB')]


In [ ]:
model_module2.getModel().getInitialStates()[species_all.index('pRB')]